In [2]:
from matplotlib import pyplot as plt
import json
import requests
import pprint as pprint
from datetime import datetime
from datetime import date
import ipywidgets as widgets
import openpyxl
from openpyxl import Workbook
import pandas as pd
from openpyxl import load_workbook
import os
import xlsxwriter
import xlrd

In [ ]:
if not os.path.exists('.variables'):
    key = input("Enter your API-key: ")
    token = input("Enter your API-token: ")
    allboards = json.loads(json.dumps(requests.get("https://api.trello.com/1/members/me/boards?key="+key+"&token="+token).json()))
    print('These are your boards: ')
    for i in allboards:
        if i['closed'] == False:
            print('Name: ',i['name'],', ID:',i['id'])
    boardid = input("Enter your board-id: ")
    
    print('These are the lists on the chosen board.')
    alllists = json.loads(json.dumps(requests.get("https://api.trello.com/1/boards/"+boardid+"/lists?key="+key+"&token="+token).json()))
    for i in alllists:
        if i['closed'] == False:
            print(i['name'])
    print()
    startlists = input("Enter the lists with cards that are not started (comma separated): ")
    print()
    blockedlists = input("Enter the lists with cards that are blocked(comma separated): ")
    print()
    doinglists = input("Enter the lists with the doing cards (comma separated): ")
    print()
    donelists = input("Enter the lists with cards that are done (comma separated): ")
    print()
    print()
    filenamewithoutextension = input("Enter the name for the excel file (without .xlsx)")
    filenamewithextension = filenamewithoutextension+".xlsx"
    print()
    print()
    file = open('.variables', 'w')
    file = open('.variables', 'a')
    file.write(key)
    file.write('\n')
    file.write(token)
    file.write('\n')
    file.write(boardid)
    file.write('\n')
    file.write(startlists)
    file.write('\n')
    file.write(blockedlists)
    file.write('\n')
    file.write(doinglists)
    file.write('\n')
    file.write(donelists)
    file.write('\n')
    file.write(filenamewithextension)
    file.close()
    print('Your variables are saved in the file \'.variables\'. Please do not delete this file.')

In [4]:
f = open('.variables', 'r')
variables = f.read().splitlines()

lijstenbeginnen = []
lijstenblocked = []
lijstendoing = []
lijstendone = []




api_key = variables[0]
api_token = variables[1]
bordid = variables[2]
lijstenbeginnen = variables[3].split(',')
lijstenblocked = variables[4].split(',')
lijstendoing = variables[5].split(',')
lijstendone = variables[6].split(',')
excelfile = variables[7]

lijstenforscrum = []
for i in lijstenbeginnen:
    lijstenforscrum.append(i)
    
lijstenforscrum.extend(lijstenblocked)
lijstenforscrum.extend(lijstendoing)
lijstenforscrum.extend(lijstendone)


f.close()

In [15]:
keys = "key="+api_key+"&token="+api_token
trello_base_url = "https://api.trello.com/1/"
board_url = trello_base_url+"boards/"+bordid
url_cards = board_url+"/cards?attachments=true&customFieldItems=true&"+keys
url_lists = board_url+"/lists?"+keys
url_customfields = board_url+"/customFields?"+keys
url_labels = board_url+"/labels?"+keys
url_members = board_url+"/members?"+keys
statussen = ['Nog starten','Blocked','Doing','Done']

In [16]:
cards = json.loads(json.dumps(requests.get(url_cards).json()))
lists = json.loads(json.dumps(requests.get(url_lists).json()))
customfields = json.loads(json.dumps(requests.get(url_customfields).json()))
labels = json.loads(json.dumps(requests.get(url_labels).json()))
members = json.loads(json.dumps(requests.get(url_members).json()))

In [7]:
customfields_dict = {}
for i in customfields:
    customfields_dict[i['id']] = {}
    if i['type'] == 'date':
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {'id': 'date'}
       

    else:
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {}
        for j in i['options']:
            customfields_dict[i['id']][i['name']]['options'][j['id']] =  j['value']['text']

customfieldsmetdate = []
for i,j in customfields_dict.items():
        for k,l in j.items():
            try:
                if l['options']['id'] == 'date':
                    customfieldsmetdate.append(i)
            except:
                pass


In [8]:
def idtodate(cardid):
    hex = cardid[0:8]
    timestamp = int(hex,16)
    timedate = datetime.fromtimestamp(timestamp)
    return timedate

In [49]:
kaarten = {}

for i in cards:
        kaarten[i['id']] = {'name': i['name'],
                            'shortLink': i['shortLink'],
                            'id': i['id'],
                            'idlist': i['idList'],
                            'customfields': i['customFieldItems'],
                            'customfieldvalues': {'Gebied': None, 
                                                  'Hoofdverantwoordelijke': None, 
                                                  'Prioriteit': None,
                                                  'Voltooid': None,
                                                  'Beginnen': None,
                                                  'Tijdsbelasting': None
                                                 },
                            'labels': {},
                            'members': {},
                            'sjabloon': i['isTemplate'],
                            'vervaldatum': None,
                            'attachments': {}

                           }
        
for i,j in kaarten.items():
    date = idtodate(i)
    j['datum aanmaak'] = str(date)
    for k in lists:
        if j['idlist'] == k['id']: j['list'] = k['name'] 
if customfields_dict == {}:
    for i,j in kaarten.items():
        j['customfieldvalues'] = {}
else:

    for i,j in kaarten.items():
        for k in j['customfields']:
        
            for l,m in customfields_dict.items():

                for n,o in m.items():
                    if k['idCustomField'] not in customfieldsmetdate:
                        for p,q in o.items():
                            for r,s in q.items():
                                if r == k['idValue']:
                                    j['customfieldvalues'][n] = s
                    if k['idCustomField'] in customfieldsmetdate:
                        j['customfieldvalues']['Beginnen'] = k['value']['date'][0:10]

                        

                  
for i in cards:
    if i['due'] != None:
        kaarten[i['id']]['vervaldatum'] = i['due'][0:10]
    for j in i['labels']:
        kaarten[i['id']]['labels'][j['name']] = j['id']


for i in cards:
    for j in i['idMembers']:

        for k in members:

            if j == k['id']:
                    kaarten[i['id']]['members'][k['id']] = k['fullName']

for i,j in kaarten.items():
    if j['list'] in lijstenbeginnen:
        j['status'] = 'Niet gestart'
    elif j['list'] in lijstendoing:
        j['status'] = 'Doing'
    elif j['list'] in lijstenblocked:
        j['status'] = 'Blocked'
    elif j['list'] in lijstendone:
        j['status'] = 'Done'

for i,j in kaarten.items():
    del j['customfields']
    del j['idlist']
    del j['id']

tedeletenlijsten = []

for i in lists:
    if i['name'] not in lijstenforscrum:
        tedeletenlijsten.append(i['name'])
tedeletenkaarten = []
for i,j in kaarten.items():
    if j['sjabloon'] == True:
        tedeletenkaarten.append(i)
    elif j['list'] in tedeletenlijsten:
        tedeletenkaarten.append(i)

for i in tedeletenkaarten:
    if i in kaarten:
        del kaarten[i]

IndentationError: expected an indented block (<ipython-input-110-1fadc7de0ffb>, line 13)

In [125]:
trellobijlagen = {}
x = 0
for i in cards:
    for j in i['attachments']:
        try:
            if j['url'][0:21]== 'https://trello.com/c/':
                tmp = j['url'][21:29]
                for k,l in kaarten.items():
                    if tmp == l['shortLink']:
                        trellobijlagen[x] = {k: l}
                        x +=1
                    
        except:
            pass

trellobijlagen
for i,j in kaarten.items():
    for k,l in trellobijlagen.items():
        for m,n in l.items():
            if i == m:
                j['attachments'] = {m:n}


{'5dc01f67a8ff3c5b3cdf7b31': {'name': 'ZaakDMS koppeling', 'shortLink': 'XtJaGKkp', 'customfieldvalues': {'Gebied': 'Algemeen', 'Hoofdverantwoordelijke': 'Gea', 'Prioriteit': '4. Laag', 'Voltooid': None, 'Beginnen': None, 'Tijdsbelasting': '3. Middel'}, 'labels': {}, 'members': {'5c89f602c36bdb8b786a431d': 'Gea de Boer', '58f5dcc3aeb458131b5d9a5f': 'Kelvin Apeldoorn'}, 'sjabloon': False, 'vervaldatum': None, 'attachments': {...}, 'datum aanmaak': '2019-11-04 12:53:59', 'list': 'Blocked', 'status': 'Blocked'}}
